# Demo of `LaPDXYTransform`

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys

plt.rcParams["figure.figsize"] = [10.5, 0.56 * 10.5]

In [ ]:
try:
    from bapsf_motion.transform import LaPDXYTransform
except ModuleNotFoundError:
    from pathlib import Path

    HERE = Path().cwd()
    BAPSF_MOTION = (HERE / ".." / ".." / ".." ).resolve()
    sys.path.append(str(BAPSF_MOTION))
    
    from bapsf_motion.transform import LaPDXYTransform

In [ ]:
tr = LaPDXYTransform(
    ("x", "y"),
    pivot_to_center=57.288,
    pivot_to_drive=134.0,
    # probe_axis_offset=10.00125,
    probe_axis_offset=20.16125,
)

In [ ]:
figwidth, figheight = plt.rcParams["figure.figsize"]
figwidth = 1.4 * figwidth
figheight = 2.0 * figheight
fig, axs = plt.subplots(2, 3, figsize=[figwidth, figheight])

axs[0,0].set_xlabel("MSpace X")
axs[0,0].set_ylabel("MSpace Y")
axs[0,1].set_xlabel("Drive X")
axs[0,1].set_ylabel("Drive Y")
axs[0,2].set_xlabel("MSpace X")
axs[0,2].set_ylabel("MSpace Y")

points = np.zeros((40, 2))
points[0:10, 0] = np.linspace(-5, 5, num=10, endpoint=False)
points[0:10, 1] = 5 * np.ones(10)
points[10:20, 0] = 5 * np.ones(10)
points[10:20, 1] = np.linspace(5, -5, num=10, endpoint=False)
points[20:30, 0] = np.linspace(5, -5, num=10, endpoint=False)
points[20:30, 1] = -5 * np.ones(10)
points[30:40, 0] = -5 * np.ones(10)
points[30:40, 1] = np.linspace(-5, 5, num=10, endpoint=False)

dpoints = tr(points, to_coords="drive")
mpoints = tr(dpoints, to_coords="motion_space")

axs[0,0].fill(points[...,0], points[...,1])
axs[0,1].fill(dpoints[...,0], dpoints[...,1])
axs[0,2].fill(mpoints[...,0], mpoints[...,1])

for pt, color in zip(
    [
        [-5, 5],
        [-5, -5],
        [5, -5],
        [5, 5],
        [0, 0]
    ],
    ["red", "orange", "green", "purple", "black"]
):
    dpt = tr(pt, to_coords="drive")
    mpt = tr(dpt, to_coords="motion_space")
    print(pt, dpt, mpt)
    axs[0,0].plot(pt[0], pt[1], 'o', color=color)
    axs[0,1].plot(dpt[..., 0], dpt[..., 1], 'o', color=color)
    axs[0,2].plot(mpt[..., 0], mpt[..., 1], 'o', color=color)

##

axs[1,0].set_xlabel("Drive X")
axs[1,0].set_ylabel("Drive Y")
axs[1,1].set_xlabel("MSpace X")
axs[1,1].set_ylabel("MSpace Y")
axs[1,2].set_xlabel("Drive X")
axs[1,2].set_ylabel("Drive Y")

points = np.zeros((40, 2))
points[0:10, 0] = np.linspace(-5, 5, num=10, endpoint=False)
points[0:10, 1] = 5 * np.ones(10)
points[10:20, 0] = 5 * np.ones(10)
points[10:20, 1] = np.linspace(5, -5, num=10, endpoint=False)
points[20:30, 0] = np.linspace(5, -5, num=10, endpoint=False)
points[20:30, 1] = -5 * np.ones(10)
points[30:40, 0] = -5 * np.ones(10)
points[30:40, 1] = np.linspace(-5, 5, num=10, endpoint=False)

mpoints = tr(points, to_coords="motion_space")
dpoints = tr(mpoints, to_coords="drive")

axs[1,0].fill(points[...,0], points[...,1])
axs[1,1].fill(mpoints[...,0], mpoints[...,1])
axs[1,2].fill(dpoints[...,0], dpoints[...,1])

for pt, color in zip(
    [
        [-5, 5],
        [-5, -5],
        [5, -5],
        [5, 5],
        [0, 0]
    ],
    ["red", "orange", "green", "purple", "black"]
):
    mpt = tr(pt, to_coords="motion_space")
    dpt = tr(mpt, to_coords="drive")
    axs[1,0].plot(pt[0], pt[1], 'o', color=color)
    axs[1,1].plot(mpt[..., 0], mpt[..., 1], 'o', color=color)
    axs[1,2].plot(dpt[..., 0], dpt[..., 1], 'o', color=color)
    print(f"X = {pt[0]}  Δ = {dpt[...,0] - pt[0]} || Y = {pt[1]}  Δ = {dpt[...,1] - pt[1]}")


### Test Transforming `drive -> motion space -> drive`

In [ ]:
mpoints = tr(points, to_coords="motion_space")
dpoints = tr(mpoints, to_coords="drive")

(
    np.allclose(dpoints, points),
    np.allclose(dpoints[...,0], points[...,0]),
    np.allclose(dpoints[...,1], points[...,1]),
    np.min(dpoints - points),
    np.max(dpoints - points),
)

In [ ]:
points = np.array([[5, 5], [5, 5]])
mpoints = tr(points, to_coords="motion_space")
dpoints = tr(mpoints, to_coords="drive")

(
    np.isclose(dpoints, points),
    np.allclose(dpoints, points),
    np.allclose(dpoints[...,0], points[...,0]),
    np.allclose(dpoints[...,1], points[...,1]),
    np.min(dpoints - points),
    np.max(dpoints - points),
)

### Test Transforming `motion space -> drive -> motion space`

In [ ]:
dpoints = tr(points, to_coords="drive")
mpoints = tr(dpoints, to_coords="motion_space")

(
    np.allclose(mpoints, points),
    np.allclose(mpoints[...,0], points[...,0]),
    np.allclose(mpoints[...,1], points[...,1]),
    np.min(mpoints - points),
    np.max(mpoints - points),
)

## Prototyping

In [ ]:
pts = [
    [-5, 5],
    [-5, -5],
    [5, -5],
    [5, 5],
    [0, 0]
]
# pts = [[-5, 5]]

pts = tr._condition_points(pts)
matrix = tr.matrix(pts, to_coords="mspace")
pts = np.concatenate(
    (pts, np.ones((pts.shape[0], 1))),
    axis=1,
)
results = np.einsum("kmn,kn->km", matrix, pts)[:-1,...]
ii = 1
# pts[ii, ...]
(pts[ii,...], results[ii,...])

In [ ]:
matrix[ii, ...]

In [ ]:
(
    pts[ii, :-1],
    tr(pts[ii, :-1], to_coords="mspace"),
)

In [ ]:
tr(pts[ii, :-1], to_coords="mspace")

## Testing Matrix Math

In [ ]:
pivot_to_center = 57.288
pivot_to_drive = 134.0
drive_polarity = np.array([1.0, 1.0])
mspace_polarity = np.array([-1.0, 1.0])

In [ ]:
def matrix_to_mspace(
    points,
    pivot_to_center,
    pivot_to_drive,
    drive_polarity,
    mspace_polarity,
):
    points = drive_polarity * points  # type: np.ndarray

    theta = np.arctan(points[..., 1] / pivot_to_drive)
    alpha = np.pi - theta

    npoints = 1 if points.ndim == 1 else points.shape[0]

    T1 = np.zeros((npoints, 3, 3)).squeeze()
    T1[..., 0, 0] = np.cos(theta)
    T1[..., 0, 2] = -pivot_to_drive * np.cos(theta)
    T1[..., 1, 0] = -np.sin(theta)
    T1[..., 1, 2] = pivot_to_drive * np.sin(theta)
    T1[..., 2, 2] = 1.0

    T2 = np.zeros((npoints, 3, 3)).squeeze()
    T2[..., 0, 0] = 1.0
    T2[..., 0, 2] = -(pivot_to_drive + pivot_to_center) * np.cos(alpha)
    T2[..., 1, 1] = 1.0
    T2[..., 1, 2] = -(pivot_to_drive + pivot_to_center) * np.sin(alpha)
    T2[..., 2, 2] = 1.0

    T3 = np.zeros((npoints, 3, 3)).squeeze()
    T3[..., 0, 0] = 1.0
    T3[..., 0, 2] = -pivot_to_center
    T3[..., 1, 1] = 1.0
    T3[..., 2, 2] = 1.0
    
    # return T1, T2, T3
    
    T_dpolarity = np.diag(drive_polarity.tolist() + [1.0])
    T_mpolarity = np.diag(mspace_polarity.tolist() + [1.0])
    
    return np.matmul(
        T_mpolarity,
        np.matmul(
            T3,
            np.matmul(
                T2,
                np.matmul(T1, T_dpolarity),
            ),
        ),
    )

In [ ]:
def matrix_to_drive(
    points,
    pivot_to_center,
    pivot_to_drive,
    drive_polarity,
    mspace_polarity,
):
    points = mspace_polarity * points  # type: np.ndarray

    # need to handle when x_L = pivot_to_center
    # since alpha can never be 90deg we done need to worry about that case
    alpha = np.arctan(points[..., 1] / (pivot_to_center + points[...,0]))

    npoints = 1 if points.ndim == 1 else points.shape[0]
    
    T1 = np.zeros((npoints, 3, 3)).squeeze()
    T1[..., 0, 0] = 1.0
    T1[..., 0, 2] = pivot_to_center
    T1[..., 1, 1] = 1.0
    T1[..., 2, 2] = 1.0

    T2 = np.zeros((npoints, 3, 3)).squeeze()
    T2[..., 0, 0] = 1.0
    T2[..., 0, 2] = -(pivot_to_drive + pivot_to_center) * np.cos(alpha)
    T2[..., 1, 1] = 1.0
    T2[..., 1, 2] = -(pivot_to_drive + pivot_to_center) * np.sin(alpha)
    T2[..., 2, 2] = 1.0
    
    T3 = np.zeros((npoints, 3, 3)).squeeze()
    T3[..., 0, 0] = 1 / np.cos(alpha)
    T3[..., 0, 2] = pivot_to_drive
    T3[..., 1, 2] = -pivot_to_drive * np.tan(alpha)
    T3[..., 2, 2] = 1.0
    
    # return T1, T2, T3
    
    T_dpolarity = np.diag(drive_polarity.tolist() + [1.0])
    T_mpolarity = np.diag(mspace_polarity.tolist() + [1.0])
    
    return np.matmul(
        T_dpolarity,
        np.matmul(
            T3,
            np.matmul(
                T2,
                np.matmul(T1, T_mpolarity),
            ),
        ),
    )

In [ ]:
def convert(
    points,
    pivot_to_center,
    pivot_to_drive,
    drive_polarity,
    mspace_polarity,
    to_coord="drive",
):
    if not isinstance(points, np.ndarray):
            points = np.array(points)
    
    if to_coord == "drive":
        matrix = matrix_to_drive(
            points,
            pivot_to_center=pivot_to_center,
            pivot_to_drive=pivot_to_drive,
            drive_polarity=drive_polarity,
            mspace_polarity=mspace_polarity,
        )
    elif to_coord == "motion_space":
        matrix = matrix_to_mspace(
            points,
            pivot_to_center=pivot_to_center,
            pivot_to_drive=pivot_to_drive,
            drive_polarity=drive_polarity,
            mspace_polarity=mspace_polarity,
        )
    else:
        raise ValueError
    
    if points.ndim == 1:
        points = np.concatenate((points, [1]))
        return np.matmul(matrix, points)[:2]

    points = np.concatenate(
        (points, np.ones((points.shape[0], 1))),
        axis=1,
    )
    
    return np.einsum("kmn,kn->km", matrix, points)[..., :2]
    

In [ ]:
point = np.array([[0, 0], [1,2], [3,4], [-1, -1]])

dpoints = convert(
    points=point,
    to_coord="drive",
    pivot_to_drive=pivot_to_drive,
    pivot_to_center=pivot_to_center,
    drive_polarity=drive_polarity,
    mspace_polarity=mspace_polarity,
)
dpoints

In [ ]:
mpoints = convert(
    points=dpoints,
    to_coord="motion_space",
    pivot_to_drive=pivot_to_drive,
    pivot_to_center=pivot_to_center,
    drive_polarity=drive_polarity,
    mspace_polarity=mspace_polarity,
)
mpoints

In [ ]:
np.isclose(mpoints, point)

In [ ]:
(mpoints - point) / point

In [ ]:
point = np.array([[0, 0], [1,2], [3,4], [-1, -1]])
# T1, T2, T3 = matrix_to_mspace(
#     points=point,
#     pivot_to_center=pivot_to_center,
#     pivot_to_drive=pivot_to_drive,
#     drive_polarity=drive_polarity,
#     mspace_polarity=mspace_polarity,
# )
T = matrix_to_mspace(
    points=point,
    pivot_to_center=pivot_to_center,
    pivot_to_drive=pivot_to_drive,
    drive_polarity=drive_polarity,
    mspace_polarity=mspace_polarity,
)
TT.shape

In [ ]:
# (
#     T1[1,...],
#     T2[1,...],
#     T3[1,...],
# )

In [ ]:
npt = np.concatenate(
    (
        point,
        np.ones((point.shape[0], 1)),
    ),
    axis=1,
)
npt

In [ ]:
# np.matmul(TT, npt, axes="(k,m,n),(k,m)->(k,n)")
np.einsum("kmn,kn->km", TT, npt)[..., :2]

In [ ]:
point

In [ ]:
P = np.diag([-1, -1, 1])
(
    P,
    np.linalg.inv(P),
)

In [ ]:
M = np.zeros((3, 3))
M[0,0] = 1
M[0,2] = -50
M[1,1] = 1
M[2,2] = 1

(
    M,
    np.linalg.inv(M),
)

In [ ]:
probe_axis_offset = 4.
pivot_to_drive = 20
pivot_to_center = 40

In [ ]:
points = np.array([
    [-5, 5],
    [-5, -5],
    [5, -5],
    [5, 5],
    [0, 0],
    [-5, 0],
    [5, 0],
])
points

In [ ]:
sine_alpha = probe_axis_offset / np.sqrt(
    pivot_to_drive**2
    + (-probe_axis_offset + points[..., 1])**2
)
alpha = np.arcsin(sine_alpha)
np.degrees(alpha)

In [ ]:
tan_beta = (-probe_axis_offset + points[..., 1]) / -pivot_to_drive
beta = np.arctan(tan_beta)
np.degrees(beta)

In [ ]:
theta = beta - alpha
theta

In [ ]:
T0 = np.zeros((points.shape[0], 3, 3)).squeeze()
T0[..., 0, 0] = np.cos(theta)
T0[..., 0, 2] = -pivot_to_center * (1 - np.cos(theta))
T0[..., 1, 0] = np.sin(theta)
T0[..., 1, 2] = pivot_to_center * np.sin(theta)
T0[..., 2, 2] = 1.0
T0[0,...]

In [ ]:
pts = np.concatenate(
    (points, np.ones((points.shape[0], 1))),
    axis=1,
)
mpoints = np.einsum("kmn,kn->km", T0, pts)[...,:-1]
mpoints

In [ ]:
tan_theta = mpoints[...,1]/(mpoints[...,0]+pivot_to_center)
theta = -np.arctan(tan_theta)
np.degrees(theta)

In [ ]:
TI = np.zeros((points.shape[0], 3, 3)).squeeze()
TI[..., 0, 2] = np.sqrt(mpoints[...,1]**2 +(pivot_to_center + mpoints[...,0])**2) - pivot_to_center
TI[..., 1, 2] = pivot_to_axis * np.tan(theta) + probe_axis_offset * (1 - (1/np.cos(theta)))
TI[..., 2, 2] = 1.0
TI[0,...]

In [ ]:
mpts = np.concatenate(
    (mpoints, np.ones((points.shape[0], 1))),
    axis=1,
)
pts = mpoints = np.einsum("kmn,kn->km", TI, mpts)[...,:-1]
pts

In [ ]:
probe_axis_offset * (1 - (1/np.cos(theta)))

In [ ]:
pivot_to_axis*np.tan(theta) + probe_axis_offset * (1 - (1/np.cos(theta)))

In [ ]:
pivot_to_axis*np.tan(theta) - probe_axis_offset * np.cos(theta) + probe_axis_offset